# ODS_project
### Optimization Methods for Recommender Systems
#### Rebecca Esegio, Giacomo Filippin, Emma Lovato
2130576, Lovato Emma, emma.lovato@studenti.unipd.it

2144564, Rebecca Esegio, rebecca.esegio@studenti.unipd.it

2130958, Giacomo Filippin, giacomo.filippin@studenti.unipd.it

## Understanding the Problem & Theoretical Foundations
The goal is to implement and compare three variants of the Frank-Wolfe algorithm for a Matrix Completion problem, a key application in recommender systems.

### The Matrix Completion Problem

We want to recover a matrix $X \in \mathbb{R}^{m \times n}$ (e.g., user ratings for movies) from which we only observe a small subset of entries. The key assumption is that the original matrix is **low-rank**. The problem can be formulated as follows:

$$
\min_{X \in \mathbb{R}^{m \times n}} f(X) := \sum_{(i,j) \in J} (X_{ij} - U_{ij})^2 \quad \text{s.t.} \quad \text{rank}(X) \le \delta
$$

where $U$ is the matrix of observed ratings, $J$ is the set of known rating indices, and $\delta$ is the desired maximum rank. Since the rank constraint is non-convex and hard to handle, a convex relaxation is used, constraining the **nuclear norm** (the sum of the singular values) of the matrix:

$$
\min_{X \in \mathbb{R}^{m \times n}} f(X) := \sum_{(i,j) \in J} (X_{ij} - U_{ij})^2 \quad \text{s.t.} \quad \|X\|_* \le \tau
$$

### Objective Function:
 $f(X)$ is a quadratic function, hence it is convex and differentiable. Its gradient is:
    $$
    \nabla f(X) = 2(X - U)_J
    $$
    where $(A)_J$ is a matrix that has the same values as $A$ on the indices $J$ and is zero elsewhere.

### Feasible Domain:
The domain $C = \{X \in \mathbb{R}^{m \times n} : \|X\|_* \le \tau\}$ is the nuclear norm ball. It is a compact and convex set. Crucially, it is the convex hull of rank-1 matrices:
    $$
    C = \text{conv}\{\tau \mathbf{u}\mathbf{v}^T : \mathbf{u} \in \mathbb{R}^m, \mathbf{v} \in \mathbb{R}^n, \|\mathbf{u}\|_2 = \|\mathbf{v}\|_2 = 1\}
    $$
    These rank-1 elements, $\tau \mathbf{u}\mathbf{v}^T$, are the **atoms** of our domain.

### Key Algorithm Components

All the Frank-Wolfe algorithms we will implement share two fundamental components: the Linear Minimization Oracle (LMO) and the Line Search.

**a) Linear Minimization Oracle (LMO)**

At each iteration $k$, the FW algorithm must solve the following linearized subproblem:
$$
\mathbf{s}_k = \text{arg min}_{\mathbf{s} \in C} \langle \mathbf{s}, \nabla f(\mathbf{X}^{(k)}) \rangle
$$
For our domain (the nuclear norm ball), the solution to this problem is found by computing the top singular vector pair of the matrix $\nabla f(\mathbf{X}^{(k)})$. If $\mathbf{u}_1, \mathbf{v}_1$ are the left and right singular vectors corresponding to the largest singular value of $\nabla f(\mathbf{X}^{(k)})$, then the solution is:
$$
\mathbf{s}_k = -\tau \cdot \mathbf{u}_1 \mathbf{v}_1^T
$$
This computation (1-SVD) is much more efficient than a full SVD, especially since the gradient is sparse.

Why We Use SVD? 
In short: We use the SVD because it is the mathematical tool that solves the Linear Minimization Oracle (LMO) when our feasible set is the nuclear norm ball.

**b) Line Search for the Optimal Step $\gamma$**

Once we find a search direction $\mathbf{d}_k$, we must choose a step-size $\gamma_k \in [0, \gamma_{max}]$ to update the solution: $\mathbf{X}^{(k+1)} = \mathbf{X}^{(k)} + \gamma_k \mathbf{d}_k$. [cite_start]Instead of using a fixed or diminishing step-size like $\gamma_k = \frac{2}{k+2}$ [cite: 429][cite_start], we can compute the optimal one with an exact line search, which for a quadratic objective function has an efficient, closed-form solution.

We need to minimize $\phi(\gamma) = f(\mathbf{X}^{(k)} + \gamma \mathbf{d}_k)$ with respect to $\gamma$:
$$
\phi(\gamma) = \|P_J(\mathbf{X}^{(k)} + \gamma \mathbf{d}_k) - P_J(U)\|_F^2 = \| (P_J(\mathbf{X}^{(k)}) - P_J(U)) + \gamma P_J(\mathbf{d}_k) \|_F^2
$$
This is a parabola in $\gamma$. To find the minimum, we set its derivative $\phi'(\gamma)$ to zero:
$$
\phi'(\gamma) = 2 \langle P_J(\mathbf{X}^{(k)}) - P_J(U), P_J(\mathbf{d}_k) \rangle_F + 2\gamma \|P_J(\mathbf{d}_k)\|_F^2 = 0
$$
Recalling that $\nabla f(\mathbf{X}^{(k)}) = 2 P_J(\mathbf{X}^{(k)} - U)$, we get:
$$
\gamma^* = - \frac{\langle \nabla f(\mathbf{X}^{(k)}), P_J(\mathbf{d}_k) \rangle_F}{2 \|P_J(\mathbf{d}_k)\|_F^2}
$$
The final step-size will be $\gamma_k = \text{max}(0, \text{min}(\gamma_{max}, \gamma^*))$.

### Understanding the Nuclear Norm Ball and Its Role in the Project

The **nuclear norm ball** is a central concept in this project. It serves as the feasible set for our optimization problem and is the primary reason why the Frank-Wolfe algorithm is such an effective tool for Matrix Completion. This section explains what it is and why it's so important.

#### 1. The Simple, Direct Definition

Simply put, the nuclear norm ball is a "container" for matrices. It is defined by two components:

* **The Nuclear Norm ($||X||_*$):** This is the sum of all the singular values of a matrix $X$. The singular values ($\sigma_i$) can be thought of as measuring the magnitude of the matrix in different principal directions. 
    $$
    \|X\|_* = \sum_i \sigma_i(X)
    $$

* **A Ball:** Like a standard Euclidean ball which contains all vectors with a norm less than a certain radius, the nuclear norm ball is the set of all matrices $X$ whose nuclear norm is less than or equal to a specified radius, $\tau$. 

Formally, the nuclear norm ball $C$ is the set:
$$
C = \{X \in \mathbb{R}^{m \times n} : \|X\|_* \le \tau \}
$$

#### 2. Why It's Crucial for Matrix Completion

The core goal of Matrix Completion is to find a **low-rank** matrix that fits the observed data.  The rank of a matrix is the number of non-zero singular values. However, directly constraining the rank (e.g., $\text{rank}(X) \le \delta$) results in a non-convex, computationally intractable optimization problem. 

This is where the nuclear norm becomes essential. It serves as a **convex relaxation** for the rank function.  The intuition comes from a powerful analogy with vectors and sparsity:

* **For Vectors (Sparsity):** In problems like LASSO, the **$l_1$ norm** ($\|\mathbf{v}\|_1 = \sum_i |v_i|$) is used as a penalty or constraint. It is well-known that minimizing the $l_1$ norm encourages solutions where many vector components are exactly zero, thus enforcing **sparsity**.

* **For Matrices (Low-Rank):** The **nuclear norm** is the matrix equivalent of the $l_1$ norm. By minimizing the sum of singular values, it encourages solutions where many singular values are zero. This enforces **low-rankness**.

Therefore, by constraining our solution $X$ to lie within a nuclear norm ball ($\|X\|_* \le \tau$), we are using a computationally feasible method to enforce the underlying assumption that the matrix we are looking for is low-rank.

#### 3. The Connection to the Frank-Wolfe Algorithm

The Frank-Wolfe algorithm is exceptionally well-suited for optimizing over the nuclear norm ball due to its geometric properties.

* **The "Atoms" are Rank-1 Matrices:** The extreme points (the "corners" or "atoms") of the nuclear norm ball are all **rank-1 matrices**.  Specifically, the feasible set $C$ is the convex hull of all rank-1 matrices with a nuclear norm of $\tau$. 

* **The LMO Finds the Best Rank-1 Matrix:** At each iteration, the Frank-Wolfe Linear Minimization Oracle (LMO) solves for the atom that best aligns with the negative gradient. For this problem, this means the LMO finds the optimal **rank-1 matrix** to move towards. 

* **Frank-Wolfe Naturally Builds a Low-Rank Solution:** Because the algorithm constructs its solution as a convex combination of the atoms it has found, the resulting matrix is inherently a sum of a few rank-1 matrices. This means the solution produced by Frank-Wolfe is naturally **low-rank**, perfectly aligning with the goal of the Matrix Completion problem.

## Python implementation
Let's translate the theory into code. We will create a modular structure to easily assemble the three algorithms.

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.sparse import csc_matrix
from sklearn.model_selection import train_test_split

# Import your custom modules
from functions import MatrixCompletionProblem
from algorithms import frank_wolfe_solver, away_step_frank_wolfe_solver, pairwise_frank_wolfe_solver

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Data Selection and Preparation
Three datasets:
1. MovieLens 2M
2. Jester Dataset 2 ?
3. Book-Crossings ?

In [ ]:
# MovieLens 2M dataset
df_ratings=pd.read_csv('./rating.csv')

reviews_groups = df_ratings.groupby("rating")["rating"].count()

plt.figure(figsize=(8, 5))
plt.bar(reviews_groups.index, reviews_groups.values, color='skyblue', edgecolor='black')
plt.title("Ratings Distribution")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.xticks(reviews_groups.index)  
plt.show()

In [ ]:
def load_movielens_100k(path_to_data='ml-100k/u.data'):
    """Loads and prepares the MovieLens 100k dataset."""
    try:
        df = pd.read_csv(path_to_data, sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
    except FileNotFoundError:
        print("Dataset not found. Please download MovieLens 100K and place u.data in a 'ml-100k' folder.")
        print("Download from: https://grouplens.org/datasets/movielens/100k/")
        return None, None

    user_map = {uid: i for i, uid in enumerate(df['user_id'].unique())}
    item_map = {iid: i for i, iid in enumerate(df['item_id'].unique())}
    
    df['user_idx'] = df['user_id'].map(user_map)
    df['item_idx'] = df['item_id'].map(item_map)
    
    num_users, num_items = len(user_map), len(item_map)
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    train_matrix = csc_matrix((train_df['rating'], (train_df['user_idx'], train_df['item_idx'])), shape=(num_users, num_items))
    test_matrix = csc_matrix((test_df['rating'], (test_df['user_idx'], test_df['item_idx'])), shape=(num_users, num_items))
                                     
    return train_matrix, test_matrix

# Load the data
train_data, test_data = load_movielens_100k()

## Running Experiments and Analyzing Results

In [ ]:
# Instantiate the problem with the training data
mc_problem = MatrixCompletionProblem(train_data)

# --- Experiment Parameters ---
TAU = 5000.0  # This is a hyperparameter to tune
MAX_ITER = 100

solvers_to_run = {
    "Classic Frank-Wolfe": frank_wolfe_solver,
    "Away-Step Frank-Wolfe": away_step_frank_wolfe_solver,
    "Pairwise Frank-Wolfe": pairwise_frank_wolfe_solver
}

results = {}

In [ ]:
for name, solver_func in solvers_to_run.items():
    start_time = time.time()
    solution_X, history = solver_func(mc_problem, tau=TAU, max_iter=MAX_ITER)
    end_time = time.time()
    
    results[name] = {
        "solution": solution_X,
        "history": history,
        "time": end_time - start_time
    }

### Plot convergence

In [ ]:
plt.figure(figsize=(12, 7))
for name, result in results.items():
    plt.plot(result['history'], label=f"{name} (time: {result['time']:.2f}s)", lw=2)

plt.xlabel("Iteration", fontsize=12)
plt.ylabel("Objective Function (Training MSE)", fontsize=12)
plt.title("Convergence of Frank-Wolfe Variants on MovieLens 100K", fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, which='both', linestyle='--')
plt.yscale('log')
plt.show()

## Analyze Results and Evaluate on Test Set

In [ ]:
## Load the others datasets


In [ ]:
# Summarize the results
summary_data = []

# Create a problem instance for the test set to calculate RMSE
test_problem = MatrixCompletionProblem(test_data)
num_test_ratings = len(test_data.data)

for name, result in results.items():
    final_train_error = result['history'][-1]
    
    # Calculate Test RMSE
    test_mse = test_problem.objective_function(result['solution'])
    test_rmse = np.sqrt(test_mse / num_test_ratings) if num_test_ratings > 0 else 0
    
    summary_data.append({
        "Algorithm": name,
        "Final Training MSE": final_train_error,
        "Test RMSE": test_rmse,
        "Time (s)": result['time']
    })

summary_df = pd.DataFrame(summary_data)
print("\n--- Experiment Summary ---")
print(summary_df.to_string(index=False))